# Compress Graph

This notebook contains code to compress our graph representation into a tiny string. This string will contain all valid edges, and only valid edges. It can be used for matching by searching from each subsequent pair of characters from a word. If all pair exists in the string then the word is accepted.


First establish the puzzle graph we will be operating over:

In [1]:

graph = {
    'b': {'r', 'l', 't', 'y', 'o'}, 
    'r': {'b', 'l', 'e', 'c', 'o'}, 
    'l': {'b', 'r', 't', 'e', 'o'}, 
    't': {'y', 'b', 'l', 'n', 'o'}, 
    'y': {'b', 't', 'n', 's', 'o'}, 
    'e': {'r', 'c', 'g', 'l', 'o'}, 
    'o': {'b', 'r', 'l', 't', 'y', 'e', 'n', 'c', 'g', 'w', 's', 'h'}, 
    'n': {'s', 'w', 'y', 't', 'o'}, 
    'c': {'r', 'e', 'g', 'h', 'o'}, 
    'g': {'h', 'w', 'e', 'c', 'o'}, 
    'w': {'s', 'n', 'g', 'h', 'o'}, 
    's': {'y', 'n', 'w', 'h', 'o'}, 
    'h': {'s', 'w', 'g', 'c', 'o'}
}

assert all(p in graph[q] for p, to in graph.items() for q in to), "graph should be symetric directed"

graph

{'b': {'l', 'o', 'r', 't', 'y'},
 'r': {'b', 'c', 'e', 'l', 'o'},
 'l': {'b', 'e', 'o', 'r', 't'},
 't': {'b', 'l', 'n', 'o', 'y'},
 'y': {'b', 'n', 'o', 's', 't'},
 'e': {'c', 'g', 'l', 'o', 'r'},
 'o': {'b', 'c', 'e', 'g', 'h', 'l', 'n', 'r', 's', 't', 'w', 'y'},
 'n': {'o', 's', 't', 'w', 'y'},
 'c': {'e', 'g', 'h', 'o', 'r'},
 'g': {'c', 'e', 'h', 'o', 'w'},
 'w': {'g', 'h', 'n', 'o', 's'},
 's': {'h', 'n', 'o', 'w', 'y'},
 'h': {'c', 'g', 'o', 's', 'w'}}

Compute the edges for the graph

In [2]:
edges = [p+q for p, out in graph.items() for q in out]

print(edges)

['by', 'bl', 'br', 'bt', 'bo', 'rc', 'rl', 're', 'rb', 'ro', 'lr', 'le', 'lb', 'lt', 'lo', 'ty', 'tl', 'tb', 'tn', 'to', 'ys', 'yb', 'yt', 'yn', 'yo', 'ec', 'el', 'er', 'eg', 'eo', 'oc', 'oy', 'ol', 'oe', 'os', 'ob', 'ot', 'on', 'og', 'ow', 'oh', 'or', 'ny', 'ns', 'nt', 'nw', 'no', 'cr', 'ce', 'cg', 'ch', 'co', 'gc', 'ge', 'gw', 'gh', 'go', 'ws', 'wn', 'wg', 'wh', 'wo', 'sy', 'sn', 'sw', 'sh', 'so', 'hc', 'hs', 'hg', 'hw', 'ho']


### Edge string examples

Here's a very inefficient example of what a graph representation will look like

In [3]:
":".join(f"{edge}" for edge in edges)

'by:bl:br:bt:bo:rc:rl:re:rb:ro:lr:le:lb:lt:lo:ty:tl:tb:tn:to:ys:yb:yt:yn:yo:ec:el:er:eg:eo:oc:oy:ol:oe:os:ob:ot:on:og:ow:oh:or:ny:ns:nt:nw:no:cr:ce:cg:ch:co:gc:ge:gw:gh:go:ws:wn:wg:wh:wo:sy:sn:sw:sh:so:hc:hs:hg:hw:ho'

A somewhat more efficience representation is one where we don't store the edge in both directions. If the code checks for character-pairs in both orders we can cut out graph representation in half.

In [4]:
undirected_edges = {''.join(sorted(edge)) for edge in edges}

":".join(f"{edge}" for edge in undirected_edges)

'eg:gw:no:co:gh:el:ny:lo:ow:hs:cg:go:ot:os:sw:by:bt:or:ty:ch:ns:nt:cr:lr:ho:hw:bo:br:sy:bl:er:ce:eo:lt:nw:oy'

### Validation

We're going to be trying various mutations on the string to shorten it, so we're going to need a way to ensure the strings we produce are valid:

In [5]:
alphabet = ''.join(chr(o) for o in range(ord('a'), ord('z')+1))
invalid_edges = [a+b for a in alphabet for b in alphabet if a+b not in edges]

assert len(invalid_edges) + len(edges) == 26**2

def validate_edge_string(string, undirected=False):
    for edge in edges:
        if edge not in string and (not undirected or edge[::-1] not in string):
            yield f"Missing edge '{edge}'"
    for edge in invalid_edges:
        if edge in string:
            yield f"Should not contain edge '{edge}'"

def is_valid_edge_string(string, undirected=False):
    return len(list(validate_edge_string(string, undirected=undirected))) == 0


valid_string = ":".join(f"{p}" for p in edges)
print(valid_string, is_valid_edge_string(valid_string))

invalid_string = ":".join(f"{p}" for p in edges[:-1])
print(invalid_string, is_valid_edge_string(invalid_string))

by:bl:br:bt:bo:rc:rl:re:rb:ro:lr:le:lb:lt:lo:ty:tl:tb:tn:to:ys:yb:yt:yn:yo:ec:el:er:eg:eo:oc:oy:ol:oe:os:ob:ot:on:og:ow:oh:or:ny:ns:nt:nw:no:cr:ce:cg:ch:co:gc:ge:gw:gh:go:ws:wn:wg:wh:wo:sy:sn:sw:sh:so:hc:hs:hg:hw:ho True
by:bl:br:bt:bo:rc:rl:re:rb:ro:lr:le:lb:lt:lo:ty:tl:tb:tn:to:ys:yb:yt:yn:yo:ec:el:er:eg:eo:oc:oy:ol:oe:os:ob:ot:on:og:ow:oh:or:ny:ns:nt:nw:no:cr:ce:cg:ch:co:gc:ge:gw:gh:go:ws:wn:wg:wh:wo:sy:sn:sw:sh:so:hc:hs:hg:hw False


### Compute the efficiency of an edge string

Efficiency is defined by the ideal number of edges (pairs of characters) divided by the actual number of edges in the string.

The ideal edge string is one where there are no unnecessary pairs. In the case of directed graph that's 72 character. In the undirected case that's 36. 

In [6]:
from collections import Counter

def efficiency(string):
    counter = Counter([p+q for p,q in zip(string, string[1:])] + [q+p for p,q in zip(string, string[1:])])
    return len(edges) / sum(counter.values())

edge_string = ":".join(f"{p}" for p in edges)
print(f"{edge_string} Efficiency: {efficiency(edge_string):.0%}")

by:bl:br:bt:bo:rc:rl:re:rb:ro:lr:le:lb:lt:lo:ty:tl:tb:tn:to:ys:yb:yt:yn:yo:ec:el:er:eg:eo:oc:oy:ol:oe:os:ob:ot:on:og:ow:oh:or:ny:ns:nt:nw:no:cr:ce:cg:ch:co:gc:ge:gw:gh:go:ws:wn:wg:wh:wo:sy:sn:sw:sh:so:hc:hs:hg:hw:ho Efficiency: 17%


### Find the shortest possible edge string


In [7]:
import random

def create_edge_string_with_random_walks(graph, max_walks=1000, undirected=True):
    """
    Randomly walk the graph recording visited edges and ensuring we don't traverse the same edge twice. Produce an
    edge string by concatonating all the walks together.
    """
    walks = []
    seen_edges = set()

    src = random.choice(list(graph))
    walk = None

    for i in range(max_walks):  
        available = [dst for dst in graph[src] if src+dst not in seen_edges]
        if not available:
            walks.append(walk)
            src = random.choice(list(graph))
            walk = None
        else:
            if not walk:
                walk = src
            dst = random.choice(available)
            walk += dst
            seen_edges.add(src + dst)
            if undirected:
                seen_edges.add(dst + src)
            src = dst
    result = ':'.join(walk for walk in walks if walk)
    
    assert is_valid_edge_string(result, undirected), \
        f"Validation failed for string '{result}': {list(validate_edge_string(result, undirected))}"
    
    return result


create_edge_string_with_random_walks(graph)

'hgchoybocrbltosytb:cerlegor:tnshwnol:gwoe:sw:ny'

In [8]:

def shorten(string, undirected=False):
    """
    Delete all characters from the string that we can while maintaining validity
    """
    keep_trying = True
    while keep_trying:
        keep_trying = False
        for i in range(len(string)):
            shortened = string[:i] + string[i+1:]
            if is_valid_edge_string(shortened, undirected=undirected):
                string = shortened
                keep_trying = True
                break
    return string


string = 'relbtnswgosytownolrceohcobroyb:lt:yn:shge:cg:wh'

len(shorten(string, undirected=True)), len(string)

(42, 47)

In [9]:
 
def find_shortest_edge_string(graph, repeats=1000, max_walks=1000, undirected=True):
    """
    Find the shortest possible edge string by repeatedly walk the graph to find candidates that are then shortened
    by deleting all unnecessary characters. 
    """
    best = None

    for _ in range(repeats):

        string = create_edge_string_with_random_walks(graph, max_walks=max_walks, undirected=undirected)
        
        shortened_string = shorten(string, undirected=undirected)
        
        if not best or len(shortened_string) < len(best):
            best = shortened_string
            print(f"Found new best: {best} (length: {len(best)}, efficiency: {efficiency(best)})")
    return best

%time compressed_edge_string = find_shortest_edge_string(graph)

compressed_edge_string


Found new best: yogecghwg:lbysntlrohswocrelonytobr:soe:wn:ch:tb (length: 47, efficiency: 0.782608695652174)
Found new best: corceonsholtynwsyos:tnwghwotbyogchgelrerblbo (length: 44, efficiency: 0.8372093023255814)
Found new best: tleoynorlobtyswgcrbysnwogerechgcotnwhsoh:lb (length: 43, efficiency: 0.8571428571428571)
Found new best: whsyonsorblegohgcotnytloerchswnybtbowgcerl (length: 42, efficiency: 0.8780487804878049)
CPU times: user 9.61 s, sys: 36.3 ms, total: 9.65 s
Wall time: 9.67 s


'whsyonsorblegohgcotnytloerchswnybtbowgcerl'